In [4]:
import hashlib
from base64 import b64encode as b64
import sys, os, pathlib
import itertools

In [5]:
def fmt_hash(typ, meth, h):
    return "%s %s:%s" % (typ, meth, h)

def print_hash(typ, meth, h, f=None):
    s = fmt_hash(typ, meth, h)
    print(s, file=f or sys.stdout)
    
def print_hex(h, meth='', f=None):
    print_hash('hex', meth, h, f)
    
def print_HEX(h, meth='', f=None):
    print_hash('HEX', meth, h, f)
    
def print_b64(h, meth='', f=None):
    print_hash('base64', meth, h, f)

In [36]:
def bhH(d):
    b64s = b64(d)
    hexs = ':'.join(hex(i)[2:] for i in d)
    HEXS = hexs.upper()
    return b64s, hexs, HEXS  

In [7]:
def calc_checksum_sha256(s):
    if not isinstance(s, bytes):
        s = s.encode('ascii')
    d = hashlib.sha256(s).digest()
    b, h, H = bhH(d)
    return d, b, h, H

def checksum_sha256(s):
    d, b, h, H = calc_checksum_sha256(s)
    m = 'sha256'
    print_hex(h, m)
    print_HEX(H, m)
    print_b64(b, m)
    return d, b, h, H

In [8]:
class UnknownMethod(ValueError):
    def __init__(self, msg):
        super().__init__("unsupported hash method: %s" % hash_method)

        
def print_checksum(b, h, H, hash_method):
    m = hash_method 
    print_hex(h, m)
    print_HEX(H, m)
    print_b64(b, m)
    
def calc_checksum(s, meth):
    d = meth(s)
    b, h, H = bhH(d)
    return d, b, h, H
        
def do_checksum(s, meth):
    if not isinstance(s, bytes):
        s = s.encode('ascii')
    d = meth(s)
    b, h, H = bhH(d)
    return d, b, h, H

def meth_lookup(hash_method):
    if hash_method=='sha256':
        meth = hashlib.sha256
    elif hash_method=='sha1':
        meth = hashlib.sha1
    else:
        raise UnknownMethod(hash_method)
    return meth
        
def checksum(ob, hash_method='sha256', display=True):
    if isinstance(ob, (str)):
        ob = ob.encode('ascii')
    elif isinstance(ob, bytes):
        pass
    elif hasattr(ob, 'read'):
        ob = ob.read()
    else:
        raise TypeError(type(ob))
        
    meth = meth_lookup(hash_method)
    
    d, b, h, H = do_checksum(ob, meth)
    if display:
        print_checksum(b, h, H, hash_method)
    return d, b, h, H

In [37]:
def pl2s(p):
    return str(p)

def get_bytes(ob):
    if isinstance(ob, bytes):
        return ob
    if isinstance(ob, str):
        return ob.encode('ascii')
    if isinstance(ob, pathlib.Path):
        with open(pl2s(ob), 'rb') as f:
            return f.read()
    if hasattr(ob, 'read'):
        return ob.read()
    raise TypeError(type(ob))
    
    
def _iter_update(n, sz):
    unit = 'bytes'
    if sz > 1024:
        sz //= 1024; n //=1024
        unit = 'KB'
    if sz > 1024:
        sz //= 1024; n //=1024
        unit = 'MB'
    print("\rReading file %d/%d %s %.1f%%         " % (n, sz, unit, n/sz*100), end="")
    
def checksum_file(file, method='sha256'):
    if isinstance(file, (str, pathlib.Path)):
        if isinstance(file, pathlib.Path):
            file = pl2s(file)
        meth = meth_lookup(method)
        with open(file, 'rb') as f:
            o = meth()
            sz = os.stat(file).st_size
            n = 0
            while True:
                b = f.read(1024*1024*10)
                if not b:
                    break
                n += len(b)
                o.update(b)
                _iter_update(n, sz)
            print("\n")
            d = o.digest()
            b, h, H = bhH(d)
            print_checksum(b, h, H, method)                
    else:
        return checksum(file, method)

In [38]:
file = "E:\\BizHawk-1.11.6\\Firmware\\GBA.BIOS"
d=checksum_file(file)

Reading file 16/16 KB 100.0%         

hex sha256:fd:25:47:72:4b:50:5f:48:7e:6d:cb:29:ec:2e:cf:f3:af:35:a8:41:a7:7a:b2:e8:5f:d8:73:50:ab:d3:65:70
HEX sha256:FD:25:47:72:4B:50:5F:48:7E:6D:CB:29:EC:2E:CF:F3:AF:35:A8:41:A7:7A:B2:E8:5F:D8:73:50:AB:D3:65:70
base64 sha256:b'/SVHcktQX0h+bcsp7C7P8681qEGnerLoX9hzUKvTZXA='


In [39]:
def compare(h1, h2, method, typ):
    match = "MATCH"
    if h1 != h2:
        match = "MISMATCH"
    print(match)
    print_hash(typ, method, h1)
    print_hash(typ, method, h2)
    def putc(c): print(c, end="")
    if h1 != h2:
        f1, f2 = fmt_hash(typ, method, h1), fmt_hash(typ, method, h2)
        for a, b in itertools.zip_longest(f1, f2, fillvalue=" "):
            if a != b:
                putc("^")
            else:
                putc(" ")
            
                
    

def compare_checksum(ob, cs, cs_type='hex', method='sha256'):
    s = get_bytes(ob)
    if method=='sha256':
        d, b, h, H = calc_checksum_sha256(s)
    else:
        raise UnknownMethod(method)
    if cs_type == 'hex':
        compare(h, cs, method, cs_type)
    elif cs_type == 'HEX':
        compare(H, cs, method, cs_type)
    elif cs_type in ('b64', 'base64'):
        compare(b, cs, method, cs_type)
    else:
        raise UnknownMethod(cs_type)
        
        
            
    

In [40]:
def test():
    p = pathlib.Path(file)
    cs = "FD2547724B505F487E6DCB29EC2ECFF3AF35A841A77Aa2E85FD87350ABD36570"
    compare_checksum(p, cs, 'HEX')

In [41]:
def main():
    fp, cs = sys.argv[1:3]
    for typ in ('HEX', 'hex', 'base64', 'HEX'):
        if typ in sys.argv:
            break
    for meth in ("sha256",):
        if meth in sys.argv:
            break
    fp = pathlib.Path(fp)
    compare_checksum(fp, cs, typ, meth)

In [16]:
sys.argv[1:3] = [p, cs]
if __name__ == '__main__':
    main()

NameError: name 'p' is not defined

In [34]:
from os import listdir as ls

p = [f for f in ls("E:\\downloads\\") if 'river' in f]
import os.path
p = os.path.join("E:\\downloads", p[0])

In [42]:
checksum_file(p, 'sha1')

Reading file 3167/3167 MB 100.0%         

hex sha1:b:cf:c5:40:19:ea:17:5b:1e:e5:1f:6d:2b:20:7a:3d:14:dd:2b:58
HEX sha1:B:CF:C5:40:19:EA:17:5B:1E:E5:1F:6D:2B:20:7A:3D:14:DD:2B:58
base64 sha1:b'C8/FQBnqF1se5R9tKyB6PRTdK1g='
